# Scenario 1 - Chicago Airbnb
You and a group of friends are considering purchasing a property in Chicago that you can use as an investment. You have heard from other people that they have made a lot of money by renting out either a room or an entire unit (apartment or house). Your friends ask you to analyze data so that they can understand how much you would charge per night based on the type of dwelling you were to purchase.

In [292]:
# importing libraries
import seaborn as sns
import pandas as pd
import joblib
from sqlalchemy import create_engine

In [293]:
# Note:: The make sure you use the information from your specific PostgreSQL installation
host = r'127.0.0.1' # denotes that the db in a local installation
db = r'MSDS610' # db we just created
user = r'postgres' # using the postgres user for this demo
pw = r'pdahal001' # this is the password established during installation
port = r'5432' # default port estabalished during install

In [294]:
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [295]:
# loading the csv file
df= pd.read_csv('listings.csv')

In [296]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6397 entries, 0 to 6396
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              6397 non-null   int64  
 1   name                            6397 non-null   object 
 2   host_id                         6397 non-null   int64  
 3   host_name                       6397 non-null   object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   6397 non-null   object 
 6   latitude                        6397 non-null   float64
 7   longitude                       6397 non-null   float64
 8   room_type                       6397 non-null   object 
 9   price                           6397 non-null   int64  
 10  minimum_nights                  6397 non-null   int64  
 11  number_of_reviews               6397 non-null   int64  
 12  last_review                     52

In [297]:
# Select 100 random rows for live data.
live_data = df[['id', 'room_type', 'price', 'number_of_reviews', 'reviews_per_month', 'availability_365']].sample(n=100, random_state=42)
live_data.to_csv("livedata.csv", index=False)


In [298]:
# Remove selected rows from main dataset
df = df.drop(live_data.index)

In [299]:
# save the updated primary DataFrame to a new CSV file
df.to_csv('updateddata.csv', index=False)

In [300]:
# Store remaining data in PostgreSQL
df.to_sql("airbnb_raw", db_conn, schema="raw", if_exists="replace", index=False)

297

# **Data Dictionary**
- `id`: Unique identifier
- `name`: Listing name
- `host_id`: Unique host identifier
- `room_type`: Type of listing (Entire home, private room, etc.)
- `price`: Price per night (Target variable)
- `number_of_reviews`: Count of reviews
- `reviews_per_month`: Average monthly reviews
- `availability_365`: Number of available days per year

# **Analytical Question**
How can we predict the price per night for an Airbnb listing based on property attributes?


In [301]:
print(df.columns)

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')


In [302]:
# Drop unnecessary columns
df = df[['room_type', 'price', 'number_of_reviews', 'reviews_per_month', 'availability_365']]

# Handle missing values
df.fillna({'reviews_per_month': 0}, inplace=True)

# Handle missing values
df.fillna({'reviews_per_month': 0}, inplace=True)

# Convert categorical variables
df = pd.get_dummies(df, columns=['room_type'], drop_first=True)

# Store cleaned data in PostgreSQL
df.to_sql("airbnb_cleaned", db_conn, schema="cleaned", if_exists="replace", index=False)

297

In [303]:
from sklearn.preprocessing import StandardScaler
import joblib
import numpy as np

In [304]:
# Feature Engineering: Create new feature - Log price
df['log_price'] = np.log1p(df['price'])

In [305]:
# Normalize numerical features
scaler = StandardScaler()
df[['number_of_reviews', 'reviews_per_month', 'availability_365']] = scaler.fit_transform(
    df[['number_of_reviews', 'reviews_per_month', 'availability_365']]
)

In [306]:
# Save scaler for future use
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [307]:
# Store engineered data
df.to_sql("airbnb_features", db_conn, schema="raw", if_exists="replace", index=False)

297

In [308]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [309]:
# Define target variable and features
X = df.drop(columns=['price', 'log_price'])
y = df['log_price']

In [310]:
# Split into training, validation, and test sets (70/15/15)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [311]:
# Train RandomForest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [312]:
# Evaluate performance
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)
print("Train RMSE:", mean_squared_error(y_train, train_pred, squared=False))
print("Validation RMSE:", mean_squared_error(y_val, val_pred, squared=False))

Train RMSE: 0.3425855109359382
Validation RMSE: 0.6142323604587694


c:\Users\prakr\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\prakr\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [313]:
# Save model
joblib.dump(model, "airbnb_model.pkl")

['airbnb_model.pkl']

In [314]:
# Load live data
live_df = pd.read_csv("livedata.csv")

In [315]:
live_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 100 non-null    int64  
 1   room_type          100 non-null    object 
 2   price              100 non-null    int64  
 3   number_of_reviews  100 non-null    int64  
 4   reviews_per_month  82 non-null     float64
 5   availability_365   100 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 4.8+ KB


In [316]:
# Handle missing values
live_df.fillna({'reviews_per_month': 0}, inplace=True)

In [317]:
# Keep 'id' column and apply one-hot encoding to 'room_type' only
live_df = pd.get_dummies(live_df, columns=['room_type'], drop_first=True)

In [318]:
# Ensure that all columns in X_train are present in live_df (add missing columns as zeros)
missing_cols = set(X_train.columns) - set(live_df.columns)
for col in missing_cols:
    live_df[col] = 0 

In [319]:
# Reorder columns to match X_train columns
live_df = live_df[X_train.columns]

In [320]:
# Scale numerical features based on the scaler used during training
scaler = joblib.load("scaler.pkl")
live_df[['number_of_reviews', 'reviews_per_month', 'availability_365']] = scaler.transform(
    live_df[['number_of_reviews', 'reviews_per_month', 'availability_365']])

In [321]:
# Load the saved model
model = joblib.load("airbnb_model.pkl")

In [322]:
# Make predictions for live data
live_predictions = model.predict(live_df)
live_df['predicted_price'] = np.expm1(live_predictions)

In [323]:
# Check column names to confirm 'price' is available
print(live_df.columns)

Index(['number_of_reviews', 'reviews_per_month', 'availability_365',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'predicted_price'],
      dtype='object')


In [331]:
print(live_df.columns)
print(df.columns)

Index(['number_of_reviews', 'reviews_per_month', 'availability_365',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'predicted_price'],
      dtype='object')
Index(['price', 'number_of_reviews', 'reviews_per_month', 'availability_365',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'log_price', 'actual_price'],
      dtype='object')


In [327]:
# Convert log_price back to the original price scale
df['actual_price'] = np.expm1(df['log_price'])

In [329]:
df.columns

Index(['price', 'number_of_reviews', 'reviews_per_month', 'availability_365',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'log_price', 'actual_price'],
      dtype='object')

In [332]:
# Merge live_df with the original df to include 'price' column
live_df = live_df.merge(df[['id', 'price']], on='id', how='left')

# Scatter Plot: Actual vs. Predicted Prices
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
sns.scatterplot(x=live_df['price'], y=live_df['predicted_price'], alpha=0.5)
plt.plot([live_df['price'].min(), live_df['price'].max()],
         [live_df['price'].min(), live_df['price'].max()],
         'k--', lw=2)  # Add a diagonal line for reference
plt.title("Actual vs Predicted Prices")
plt.xlabel("Actual Price")
plt.ylabel("Predicted Price")
plt.show()


KeyError: "['id'] not in index"

In [ ]:
print(live_df.columns)

Index(['number_of_reviews', 'reviews_per_month', 'availability_365',
       'room_type_Hotel room', 'room_type_Private room',
       'room_type_Shared room', 'predicted_price'],
      dtype='object')
